In [1]:
import json
import re
import random
import datetime, hashlib
import utils

In [2]:
def whitespace_tokenize(text):
    text = text.strip()
    if not text:
        return []
    tokens = text.split()
    return tokens


In [3]:
def prepro(*argvs):
    processed_texts = []
    for argv in argvs:
        utext = utils.strQ2B(argv)
        utext = " ".join(whitespace_tokenize(utext))
        processed_texts.append(utext)
    return processed_texts

In [4]:
# 非事实类数据转化为SQuAD
# 目前无法处理answer为空的情况
# coding:utf-8
from tqdm import tqdm

input_data = []
with open('./data/CIPS-sogou/train.v1.json', "r") as reader:
    for line in reader:
        result = json.loads(line.strip())
        input_data.append(result)
output_data = {}
output_data["version"] = "v1.3"
output_data["data"] = []
md5_f = hashlib.md5
max_seq_length = 300
counter_freq = {}
for entry in tqdm(input_data):
#     pbar.set_description("has finished {} records".format(len(output_data["data"])))
    qas_id = entry["query_id"]
    question_text = entry["query"]
    for paragraph in entry["passages"]:
        title = ""
        record_id = qas_id # 同样得问题对应于同一个id
        squad_dict = {"title": title, "id": record_id, "paragraphs": []}
        
        find_no_answer_from_passage = False
        beyond_max_text_length = False
        passage_text = paragraph["passage_text"]
        passage_id = paragraph["passage_id"]
        for answer in entry["answer"]:
            answer_id = answer["answer_id"]
            from_passage = answer["from_passage"]
            if from_passage == passage_id:
                answer_text, paragraph_text = prepro(answer["answer_text"], paragraph["passage_text"])
                if len(paragraph_text) > max_seq_length:
                    beyond_max_text_length = True
                    break
                if not re.search(re.escape(answer_text), paragraph_text):
                    find_no_answer_from_passage = True
                    break
                answer_start = re.search(re.escape(answer_text), paragraph_text).span()[0]
                paragraphs_id = "{}-{}".format(record_id, passage_id)
                context = paragraph_text
                qas_list = []
                qas_list.append({"id": "{}-{}-{}".format(record_id, from_passage, answer_id), "question": question_text, 
                                 "answers":[{"id":1, "text": answer_text, "answer_start": answer_start}]})
                squad_dict["paragraphs"].append({"id": paragraphs_id, "context": context, "qas": qas_list})
                break
        
        if find_no_answer_from_passage or beyond_max_text_length:
            continue
        output_data["data"].append(squad_dict)
print("append %d records to file" % len(output_data["data"]))
with open("./data/CIPS-sogou/train.v1.squad.json", "w") as writer:
    writer.write(json.dumps(output_data))
print("finished")

100%|██████████| 30000/30000 [00:51<00:00, 579.33it/s]


append 187080 records to file
finished


In [5]:
with open("./data/CIPS-sogou/train.v1.squad.json", "r") as reader:
    results = json.load(reader)["data"]
for result in results:
    import pdb;pdb.set_trace()
    print(result)

> <ipython-input-5-b57e2a2a7e7c>(5)<module>()
-> print(result)
(Pdb) q


BdbQuit: 

In [ ]:
with open("./data/DRCD/DRCD_dev.json", "r", encoding="utf-8") as reader:
    input_data = json.load(reader)["data"]
for entry in input_data:
    print(entry)
    import pdb; pdb.set_trace()

In [ ]:
with open("./data/CIPS-sogou/valid.squad.json", "r", encoding="utf-8") as reader:
    results = json.load(reader, encoding="utf-8")
print(json.dumps(results, indent=4))

In [ ]:
results_list = sorted(results.items(), key=lambda x: x[0])
for key, value in results_list:
    print(key + '' + value)

In [ ]:
%run ./data/SQuAD1.1/evaluate-v1.1.py ./data/CIPS-sogou/valid.squad.json ./checkpoints/CIPS-sogou-unfactoid/predictions.json